In [1]:
import pandas as pd
import pathlib as pl
import subprocess

In [2]:
dry_run = True

In [3]:
def in_notebook():
    try:
        from IPython import get_ipython
        if 'IPKernelApp' not in get_ipython().config:  # pragma: no cover
            return False
    except ImportError:
        return False
    except AttributeError:
        # In a terminal/shell, get_ipython() will return None
        return False
    return True

In [4]:
here = pl.Path.cwd() if in_notebook() else pl.Path(__file__).parent

fig_version_path = here / 'FIG_VERSION'
with fig_version_path.open('r') as file:
    version = file.read().strip()
    
alias_table = pd.read_csv(here / 'figure_aliases.csv')

science_to_alias = alias_table.set_index('science_submission')['alias'].to_dict()
alias_to_science = alias_table.set_index('alias')['science_submission'].to_dict()
nature_to_alias = alias_table.set_index('nature_submission')['alias'].to_dict()
alias_to_nature = alias_table.set_index('alias')['nature_submission'].to_dict()

parent_directories = list(here.glob('fig_*'))
contents = list(here.glob('fig_*/*'))

In [10]:
bad_files = []

def initial_rename_directory(d):
    nature_name = d.name.removeprefix('fig_')
    nature_name = d.name.split('_')[1].upper()
    alias = nature_to_alias[nature_name]
    new_name = f'fig_{nature_name}_{alias}'
    git_mv_cmd = ['git', 'mv', str(d), str(here / new_name)]
    print(' '.join(git_mv_cmd))
    if not dry_run:
        subprocess.run(git_mv_cmd)
        
    return new_name
        
def initial_rename_file(f):
    science_name_parts = f.stem.removeprefix('Fig').removeprefix('_').split('_')
    science_name_parts[1] = science_name_parts[1]
    if f.suffix in ['.png', '.svg']:
        extra = ''
        science_name = ''.join(science_name_parts)
    elif f.suffix == '.ipynb':
        extra = '__'+'_'.join(science_name_parts[1:])
        science_name = science_name_parts[0].upper()
    else:
        raise ValueError(f"Unsupported file type: {f.suffix}")
        
    if science_name not in science_to_alias:
        print(f"Warning: {science_name} for file {f.name} not found in alias table.")
        bad_files.append(f)
        return
        
    alias = science_to_alias[science_name]
    nature_name = alias_to_nature[alias]
    new_name = f'fig_{nature_name}__{alias}{extra}'
    git_mv_cmd = ['git', 'mv', str(f), str(f.with_stem(new_name))]
    print(' '.join(git_mv_cmd))
    if not dry_run:
        subprocess.run(git_mv_cmd)
        
for d in parent_directories:
    assert d.is_dir(), f"{d} is not a directory"
    # if 'fig_4' not in d.name:
    #     continue
    new_dir_name = initial_rename_directory(d)
    
    if not dry_run:
        d = d.with_name(new_dir_name)

    for ext in ['.png', '.svg', '.ipynb']:
        for f in d.rglob(f'*{ext}'):
            if not f.is_file():
                continue
            initial_rename_file(f)
    print()
    
# Check for any bad files that were not processed
if bad_files:
    print("The following files were not processed due to missing aliases:")
    for bf in bad_files:
        print(bf.name)

git mv /home/john/projects/can-manuscript/fig_s5 /home/john/projects/can-manuscript/fig_S5_crimson_logic
git mv /home/john/projects/can-manuscript/fig_s5/graphics/Fig_6_D.png /home/john/projects/can-manuscript/fig_s5/graphics/fig_S5D__solid_air.png
git mv /home/john/projects/can-manuscript/fig_s5/graphics/Fig_6_B.png /home/john/projects/can-manuscript/fig_s5/graphics/fig_S5B__boiling_ice.png
git mv /home/john/projects/can-manuscript/fig_s5/graphics/Fig_6_C.png /home/john/projects/can-manuscript/fig_s5/graphics/fig_S5C__liquid_rock.png
git mv /home/john/projects/can-manuscript/fig_s5/graphics/Fig_6_A.png /home/john/projects/can-manuscript/fig_s5/graphics/fig_S5A__frozen_fire.png
git mv /home/john/projects/can-manuscript/fig_s5/graphics/Fig_6_D.svg /home/john/projects/can-manuscript/fig_s5/graphics/fig_S5D__solid_air.svg
git mv /home/john/projects/can-manuscript/fig_s5/graphics/Fig_6_C.svg /home/john/projects/can-manuscript/fig_s5/graphics/fig_S5C__liquid_rock.svg
git mv /home/john/proje